In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from timm import create_model
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'Genus']]
df.head(2)

,ImageFilePath,Genus
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = create_model("convnext_base", pretrained=True).to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_image_features(image_path) : 
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model.forward_features(image_tensor).mean(dim=[2, 3])

    return features.cpu().numpy()
    

In [6]:
X = np.vstack([extract_image_features(img) for img in tqdm(df["ImageFilePath"], desc="Processing ... ") ])

print(f"Extracted feature shape: {X.shape}")

le = LabelEncoder()

y = le.fit_transform(df["Genus"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Processing ... : 100%|██████████| 11399/11399 [21:09<00:00,  8.98it/s]

Extracted feature shape: (11399, 1024)


In [7]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 42


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=150, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 71.58% | Prec: 78.58% | Rec: 71.58% | F1: 73.53% | Bal Acc: 68.56% | MCC: 0.6896
LogisticRegression        | Acc: 95.61% | Prec: 95.67% | Rec: 95.61% | F1: 95.61% | Bal Acc: 85.26% | MCC: 0.9505
SVMLinear                 | Acc: 97.02% | Prec: 96.93% | Rec: 97.02% | F1: 96.90% | Bal Acc: 87.53% | MCC: 0.9664
SVMPolynomial             | Acc: 94.12% | Prec: 93.50% | Rec: 94.12% | F1: 93.53% | Bal Acc: 67.37% | MCC: 0.9337
SVMRadialBasis            | Acc: 92.19% | Prec: 91.58% | Rec: 92.19% | F1: 91.40% | Bal Acc: 65.16% | MCC: 0.9120
NearestNeighbor           | Acc: 88.38% | Prec: 88.16% | Rec: 88.38% | F1: 87.89% | Bal Acc: 73.98% | MCC: 0.8689
RandomForest              | Acc: 85.79% | Prec: 85.44% | Rec: 85.79% | F1: 84.47% | Bal Acc: 55.02% | MCC: 0.8396
MLP_Baseline              | Acc: 97.02% | Prec: 96.95% | Rec: 97.02% | F1: 96.95% | Bal Acc: 87.52% | MCC: 0.9664


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Calathus,Synuchus,Calathus,Calathus,Calathus,Calathus,Calathus,Synuchus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Synuchus,Agonum,Agonum,Synuchus,Synuchus,Synuchus,Calathus,Agonum


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/16.BioCLIP-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/16.BioCLIP-linear-probing-genus-metrics.csv", index=False)